# Fulltext Context Provider (Optional)

In this notebook, you'll use a **context provider** to automatically inject knowledge graph context before agent invocations. Unlike tools that the agent must explicitly call, context providers run automatically — ensuring relevant information is always available.

Review and run the code.

***

Load the environment variables and import the required Python modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

from agent_framework.azure import AzureAIClient
from agent_framework_neo4j import Neo4jContextProvider, Neo4jSettings
from azure.identity.aio import AzureCliCredential

from config import get_neo4j_driver, get_agent_config

Get the agent configuration and load Neo4j settings from the environment.

In [ ]:
# Get configuration from environment
config = get_agent_config()

# Load Neo4j settings
neo4j_settings = Neo4jSettings()

print(f"Neo4j URI: {neo4j_settings.uri}")
print(f"Fulltext Index: {neo4j_settings.fulltext_index_name}")

## Create the Context Provider

Create a `Neo4jContextProvider` with fulltext search. Unlike tools that the agent must explicitly call, context providers run automatically before each agent invocation.

The provider will:
1. Extract the user's query from the conversation
2. Search the Neo4j fulltext index for matching content
3. Inject the results into the agent's context

In [ ]:
# Create context provider with fulltext search
provider = Neo4jContextProvider(
    uri=neo4j_settings.uri,
    username=neo4j_settings.username,
    password=neo4j_settings.get_password(),
    index_name=neo4j_settings.fulltext_index_name,
    index_type="fulltext",
    top_k=3,
    context_prompt=(
        "## Knowledge Graph Context\n"
        "Use the following information from the knowledge graph "
        "to answer questions about companies, products, and financials:"
    ),
)

> The `context_prompt` is prepended to the search results. The `top_k` parameter controls how many results are returned.

***

## Create and Run the Agent

Create an agent with the context provider. The agent doesn't need any tools — the provider handles knowledge retrieval automatically. The agent is created once and reused for all queries below.

In [ ]:
# Connect provider and create agent
credential = AzureCliCredential()
await provider.__aenter__()
print("Connected to Neo4j!\n")

client = AzureAIClient(
    project_endpoint=config.project_endpoint,
    model_deployment_name=config.model_name,
    async_credential=credential,
)
await client.__aenter__()

agent = await client.create_agent(
    name="workshop-fulltext-agent",
    instructions=(
        "You are a helpful assistant that answers questions about companies "
        "using the provided knowledge graph context. Be concise and cite "
        "specific information from the context when available."
    ),
    context_providers=[provider],
).__aenter__()

session = agent.create_session()

async def ask(query):
    """Run a query against the agent and print the response."""
    print(f"User: {query}\n")
    print("Assistant: ", end="", flush=True)
    response = await agent.run(query, session=session)
    print(response.text)
    print()

print(f"Agent '{agent.name}' created")

In [ ]:
await ask("What products does Microsoft offer?")

> The context provider automatically searched the fulltext index and injected relevant results before the LLM generated its response. The agent didn't need to decide whether to call a tool — the context was always available.

***

Run the cells below to experiment with different queries. You can also modify the queries or add new cells.

***

[View the complete code](../financial_data_load/solution_srcs/04_01_fulltext_context_provider.py)

[Move on to Lab 7 - Agent Memory](../Lab_7_Agent_Memory)

In [ ]:
await ask("Tell me about risk factors for technology companies")

In [ ]:
await ask("What are some financial metrics mentioned in SEC filings?")

In [ ]:
# Cleanup
await agent.__aexit__(None, None, None)
await client.__aexit__(None, None, None)
await provider.__aexit__(None, None, None)
await credential.close()